# Wrapping the call for the Bayesian optimization loop

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
import os
import numpy as np
import netCDF4 as nc
import xarray as xr
import datatree as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sithom.plot import plot_defaults, label_subplots, lim
from tcpips.constants import DATA_PATH, FIGURE_PATH
from adforce.mesh import xr_loader

plot_defaults()

In [3]:
from slurmpy import Slurm

In [11]:
s = Slurm("nws13", 
          {"nodes": 1, 
           "account": "n01-SOWISE", 
           "partition": "standard", 
           "qos": "standard", 
           "time": "1:0:0", 
           "tasks-per-node": 128,
           "cpus-per-task": 1, 
           "output": "test.out", 
           "error": "test.out",
           "mail-type": "ALL",
           "mail-user": "sdat2@cam.ac.uk"}
)
jid = s.run(
"""
module load PrgEnv-gnu/8.3.3
module load cray-hdf5-parallel
module load cray-netcdf-hdf5parallel

work=/mnt/lustre/a2fs-work1/work/n01/n01/sithom
source $work/.bashrc

conda activate base

python -c "import numpy as np; print('numpy', np.__version__)"
python -c "import netCDF4 as nc; print('netCDF4', nc.__version__)"
python -c "import xarray as xr; print('xarray', xr.__version__)"
python -c "import cartopy; print('cartopy', cartopy.__version__)"
"""
)
jid

b'Submitted batch job 5751164'


5751164

In [12]:
ls

comp.ipynb  f63.ipynb  mesh.ipynb  slurm-scripts/  test.out     wrap.ipynb
f22.ipynb   logs/      README.md   test.err        tides.ipynb
